In [ ]:
%matplotlib inline
from pandas.plotting import register_matplotlib_converters

register_matplotlib_converters()

import warnings

warnings.filterwarnings("ignore")

## Data Access

### servers, servers everywhere and not a bit to flip

![](https://imgs.xkcd.com/comics/digital_data.png)

## whoami

`ocefpaf` (Filipe Fernandes)

- Physical Oceanographer
- Data Plumber
- Code Janitor
- CI babysitter
- Amazon-Dash-Button for conda-forge

## My day job: IOOS


![](https://raw.githubusercontent.com/ocefpaf/2018-SciPy-talk/gh-pages/images/IOOS-RAs.jpg)

## Big or small we need data!

- There are various sources: variety of servers, APIs, and web services. Just to list a few: OPeNDAP, ERDDAP, THREDDS, ftp, http(s), S3, LAS, etc.

![](https://imgs.xkcd.com/comics/data_pipeline.png)

## Web Services/Type of servers

| Data Type                              | Web Service                       | Response    |
|----------------------------------------|-----------------------------------|-------------|
| In-situ data<br>(buoys, stations, etc) | OGC SOS                           | XML/CSV     |
| Gridded data (models, satellite)       | OPeNDAP                           | Binary      |
| Raster Images                          | OGC WMS                           | GeoTIFF/PNG |
| ERDDAP                                 | Restful API                       | *           |

* You imagination is the limit!

## What are we going to see in this tutorial?
### Browse and access data from:

1. ERDDAP
2. OPeNDAP
3. SOS
4. WMS
5. CSW\*
* Not really a server but a "server" aggregator.

## 1) ERDDAP

### Learning objectives:

- Explore an ERDDAP server with the python interface (erddapy);
- Find a glider for a time/region of interest;
- Download the data with a familiar format and create some plots.

## What is ERDDAP?

- Flexible outputs: .html table, ESRI .asc and .csv, .csvp, Google Earth .kml, OPeNDAP binary, .mat, .nc, ODV .txt, .tsv, .json, and .xhtml
- RESTful API to access the data
- Standardize dates and time in the results
- Server-side searching and slicing

In [ ]:
from erddapy import ERDDAP

e = ERDDAP(
    server="https://data.ioos.us/gliders/erddap",
    protocol="tabledap"
)

### What services are available in the server?

In [ ]:
import pandas as pd

df = pd.read_csv(
    e.get_search_url(response="csv", search_for="all")
)

In [ ]:
print(
    f'We have {len(set(df["tabledap"].dropna()))} '
    f'tabledap, {len(set(df["griddap"].dropna()))} '
    f'griddap, and {len(set(df["wms"].dropna()))} wms.'
)

### Let's query all the datasets that have the "trajectoryprofile" CDM data type.

In [ ]:
url = e.get_categorize_url(
    categorize_by="cdm_data_type",
    value="trajectoryprofile",
    response="csv",
)

df = pd.read_csv(url)
dataset_ids = df.loc[
    ~df["tabledap"].isnull(), "Dataset ID"
].tolist()

dataset_ids_list = "\n".join(dataset_ids)
print(f"Found {len(dataset_ids)} datasets")

### Let us narrow our search to deployments that within a lon/lat/time extent.

In [ ]:
from ipyleaflet import Map, FullScreenControl, Rectangle

min_lon, max_lon = -72, -69
min_lat, max_lat = 38, 41

rectangle = Rectangle(
    bounds=((min_lat, min_lon), (max_lat, max_lon))
)

m = Map(
    center=((min_lat + max_lat) / 2,
            (min_lon + max_lon) / 2), zoom=6
)
m.add_layer(rectangle)
m.add_control(FullScreenControl())

In [ ]:
m

In [ ]:
kw = {
    "min_time": "2016-07-10T00:00:00Z",
    "max_time": "2017-02-10T00:00:00Z",
    "min_lon": min_lon,
    "max_lon": max_lon,
    "min_lat": min_lat,
    "max_lat": max_lat,
}

In [ ]:
search_url = e.get_search_url(response="csv", **kw)
search = pd.read_csv(search_url)
dataset_ids = search["Dataset ID"].values

dataset_ids_list = "\n".join(dataset_ids)

In [ ]:
print(f"Found {len(dataset_ids)} Datasets:\n{dataset_ids_list}")

In [ ]:
e.dataset_id = "whoi_406-20160902T1700"

e.variables = [
    "depth",
    "latitude",
    "longitude",
    "salinity",
    "temperature",
    "time",
]

url = e.get_download_url()
print(url)

In [ ]:
import pandas as pd

df = e.to_pandas(
    index_col="time (UTC)",
    parse_dates=True
).dropna()
df.head()

Exercise: experiment with the `e.to_xarray()` method. Think about why/where use one or the other?

In [ ]:
import cartopy.crs as ccrs
import matplotlib.pyplot as plt


def plot_track(df):
    x = df["longitude (degrees_east)"]
    y = df["latitude (degrees_north)"]
    dx, dy = 2, 4

    fig, ax = plt.subplots(
        figsize=(9, 9), subplot_kw={"projection": ccrs.PlateCarree()}
    )
    cs = ax.scatter(
        x, y, c=df["temperature (Celsius)"], s=50, alpha=0.5, edgecolor="none"
    )
    cbar = fig.colorbar(
        cs, orientation="vertical", fraction=0.1, shrink=0.9, extend="both"
    )
    ax.coastlines("10m")
    ax.set_extent([x.min() - dx, x.max() + dx, y.min() - dy, y.max() + dy])
    return fig, ax

In [ ]:
fig, ax = plot_track(df)

In [ ]:
import matplotlib.dates as mdates
import matplotlib.pyplot as plt
import palettable
from palettable.cmocean.sequential import Thermal_20

cmap = Thermal_20.mpl_colormap


def plot_glider(df):
    fig, ax = plt.subplots(figsize=(17, 2))
    cs = ax.scatter(
        df.index,
        df["depth (m)"],
        s=15,
        c=df["temperature (Celsius)"],
        marker="o",
        edgecolor="none",
        cmap=cmap,
    )

    ax.invert_yaxis()
    xfmt = mdates.DateFormatter("%H:%Mh\n%d-%b")
    ax.xaxis.set_major_formatter(xfmt)

    cbar = fig.colorbar(cs, orientation="vertical", extend="both")
    cbar.ax.set_ylabel("Temperature ($^\circ$C)")
    ax.set_ylabel("Depth (m)")
    return fig, ax

In [ ]:
fig, ax = plot_glider(df)

In [ ]:
import gsw
import numpy as np


def plot_ts():
    fig, ax = plt.subplots(figsize=(5, 5))

    s = np.linspace(0, 42, 100)
    t = np.linspace(-2, 40, 100)

    s, t = np.meshgrid(s, t)
    sigma = gsw.sigma0(s, t)

    cnt = np.arange(-7, 40, 5)
    cs = ax.contour(s, t, sigma, colors="gray", levels=cnt)
    ax.clabel(cs, fontsize=9, inline=1, fmt="%2i")

    ax.set_xlabel("Salinity [g kg$^{-1}$]")
    ax.set_ylabel("Temperature [$^{\circ}$C]")
    ax.scatter(df["temperature (Celsius)"], df["salinity (1)"], s=10, alpha=0.25)

    ax.grid(True)
    ax.axis([9, 23, 28, 37.5])
    return fig, ax

In [ ]:
plot_ts();

In [ ]:
responses = ["mat", "json", "ncCF", "ncCFHeader"]

for response in responses:
    print(f"{e.get_download_url(response=response)}\n")

Exercise: explore the web interface for the OOI server URL:

https://erddap-uncabled.oceanobservatories.org/uncabled/erddap

or the IOOS glider dac:

https://gliders.ioos.us/erddap

and find a dataset of interested, download a format that you are familiar with and plot it (using the web interface or the Python, your choice).

## 2) OPeNDAP

### Learning objectives:

- Open model data from a THREDDS server via OPeNDAP with `xarray`;
- Discuss the differences with an `erddapy` request;
- Plot it using `xarray` interface.

In [ ]:
import xarray as xr

url = (
    "http://oceanmodeling.pmc.ucsc.edu:8080/thredds/dodsC/"
    "ccsra_2016a_phys_agg_zlevs/fmrc/"
    "CCSRA_2016a_Phys_ROMS_z-level_(depth)_Aggregation_best.ncd"
)
ds = xr.open_dataset(url)
ds

In [ ]:
selection = ds.sel(time="2017-01-01", z=-2.0)
selection

In [ ]:
selection["temp"].plot(figsize=(6, 6));

## 3) SOS

### Learning objectives:

- Quick explanation on the SOS API.

In [ ]:
url = (
    "https://opendap.co-ops.nos.noaa.gov/ioos-dif-sos/SOS?"
    "service=SOS"
    "&request=GetObservation"
    "&version=1.0.0"
    "&observedProperty=water_surface_height_above_reference"
    "&offering=urn:ioos:station:NOAA.NOS.CO-OPS:8454000"
    "&responseFormat=text/csv"
    "&eventTime=2018-07-04T00:00:00Z/2018-07-05T00:00:00Z"
    "&result=VerticalDatum==urn:ogc:def:datum:epsg::5103"
    "&dataType=PreliminarySixMinute"
)

In [ ]:
from datetime import datetime, timedelta

request = "GetObservation"
version = "1.0.0"
variable = "water_surface_height_above_reference_datum"
buoy = "8771450"
response = "text/csv"
vdatum = "urn:ogc:def:datum:epsg::5103"
data_type = "PreliminarySixMinute"

today = datetime.today()
yesterday = today - timedelta(days=1)

In [ ]:
url = (
    f"https://opendap.co-ops.nos.noaa.gov/ioos-dif-sos/SOS?"
    f"service=SOS&request={request}"
    f"&version={version}"
    f"&observedProperty={variable}"
    f"&offering=urn:ioos:station:NOAA.NOS.CO-OPS:8454000"
    f"&responseFormat={response}"
    f"&eventTime={yesterday:%Y-%m-%dT%H:%M:%SZ}/"
    f"{today:%Y-%m-%dT%H:%M:%SZ}"
    f"&result=VerticalDatum=={vdatum}"
    f"&dataType={data_type}"
)

print(url)

In [ ]:
import pandas as pd

df = pd.read_csv(url, index_col="date_time", parse_dates=True)
df.head()

In [ ]:
col = df.columns[df.columns.str.startswith(variable)]
ax = df[col].plot.line(legend=False)
ax.grid(True)
ax.set_title(col.values[0]);

In [ ]:
from ipyleaflet import FullScreenControl, Map, Marker

location = df["latitude (degree)"].unique()[0], df["longitude (degree)"].unique()[0]
m = Map(center=location, zoom_start=12)
marker = Marker(location=location)

m.add_layer(marker)
m.add_control(FullScreenControl())

In [ ]:
m

## 4) WMS

### Learning objectives:

- Add a WMS layer to an interactive map. ("Hurricane viz widget.")

In [ ]:
from ipyleaflet import Map, WMSLayer, basemaps, FullScreenControl
from ipywidgets import SelectionSlider
from traitlets import Unicode

time_options = [
    "13:00", "13:30", "14:00", "14:30", "15:00", "15:30",
    "16:00", "16:30", "17:00", "17:30", "18:00", "18:30",
]

slider = SelectionSlider(description="Time:", options=time_options)

def update_wms(change):
    time_wms.time = "2020-07-25T{}".format(slider.value)

slider.observe(update_wms, "value")

class TimeWMSLayer(WMSLayer):

    time = Unicode("").tag(sync=True, o=True)

In [ ]:
time_wms = TimeWMSLayer(
    url="https://mesonet.agron.iastate.edu/cgi-bin/wms/nexrad/n0r-t.cgi?",
    layers="nexrad-n0r-wmst",
    time="2020-07-25T13:00:00Z",
    format="image/png",
    transparent=True,
    attribution="Weather data © 2012 IEM Nexrad",
)
m = Map(basemap=basemaps.CartoDB.Positron, center=(30, -88), zoom=5)
m.add_layer(time_wms)
m.add_control(FullScreenControl())

In [ ]:
m

In [ ]:
slider

## 5) Catalog Service Web (CSW)
### Is there a canonical source for data?

![](https://i.kym-cdn.com/photos/images/newsfeed/001/093/557/142.gif)

Well, kind of... The closet thing is the IOOS CSW catalog.

For more info please see [https://data.ioos.us/](https://data.ioos.us/)

## Catalog Service for the Web (CSW)

- A single source to find endpoints
- Nice python interface:<br>`owslib.csw.CatalogueServiceWeb`
- Advanced filtering:<br>`owslib.fes`

![](https://raw.githubusercontent.com/ocefpaf/2018-SciPy-talk/gh-pages/images/IOOS.svg)

In [ ]:
from datetime import datetime

min_lat, max_lat = 20, 30
min_lon, max_lon = -82, -97
bbox = [min_lon, min_lat, max_lon, max_lat]
crs = "urn:ogc:def:crs:OGC:1.3:CRS84"

# Temporal range of 1 week.
start = datetime(2017, 4, 14, 0, 0, 0)
stop = datetime(2017, 4, 21, 0, 0, 0)

# Sea surface temperature CF names.
cf_names = [
    "sea_water_temperature",
    "sea_surface_temperature",
    "sea_water_potential_temperature",
    "equivalent_potential_temperature",
    "sea_water_conservative_temperature",
    "pseudo_equivalent_potential_temperature",
]

In [ ]:
def fes_date_filter(start, stop, constraint="overlaps"):
    from owslib import fes

    start = start.strftime("%Y-%m-%d %H:00")
    stop = stop.strftime("%Y-%m-%d %H:00")
    if constraint == "overlaps":
        propertyname = "apiso:TempExtent_begin"
        begin = fes.PropertyIsLessThanOrEqualTo(propertyname=propertyname, literal=stop)
        propertyname = "apiso:TempExtent_end"
        end = fes.PropertyIsGreaterThanOrEqualTo(
            propertyname=propertyname, literal=start
        )
    elif constraint == "within":
        propertyname = "apiso:TempExtent_begin"
        begin = fes.PropertyIsGreaterThanOrEqualTo(
            propertyname=propertyname, literal=start
        )
        propertyname = "apiso:TempExtent_end"
        end = fes.PropertyIsLessThanOrEqualTo(propertyname=propertyname, literal=stop)
    else:
        raise NameError(f"Unrecognized constraint {constraint}")
    return begin, end

In [ ]:
from owslib import fes

kw = dict(wildCard="*", escapeChar="\\", singleChar="?", propertyname="apiso:AnyText")

or_filt = fes.Or([fes.PropertyIsLike(literal=("*%s*" % val), **kw) for val in cf_names])

begin, end = fes_date_filter(start, stop)
bbox_crs = fes.BBox(bbox, crs=crs)

filter_list = [
    fes.And(
        [
            bbox_crs,
            begin,
            end,
            or_filt,
            fes.Not([fes.PropertyIsLike(literal="*cdip*", **kw)]),
        ]
    )
]

In [ ]:
def get_csw_records(csw, filter_list, pagesize=10, maxrecords=1000):
    """Iterate `maxrecords`/`pagesize` times until the requested value in
    `maxrecords` is reached.
    """
    from owslib.fes import SortBy, SortProperty

    # Iterate over sorted results.
    sortby = SortBy([SortProperty("dc:title", "ASC")])
    csw_records = {}
    startposition = 0
    nextrecord = getattr(csw, "results", 1)
    while nextrecord != 0:
        csw.getrecords2(
            constraints=filter_list,
            startposition=startposition,
            maxrecords=pagesize,
            sortby=sortby,
        )
        csw_records.update(csw.records)
        if csw.results["nextrecord"] == 0:
            break
        startposition += pagesize + 1  # Last one is included.
        if startposition >= maxrecords:
            break
    csw.records.update(csw_records)

In [ ]:
from owslib.csw import CatalogueServiceWeb

endpoint = "https://data.ioos.us/csw"
csw = CatalogueServiceWeb(endpoint, timeout=60)

get_csw_records(csw, filter_list, pagesize=10, maxrecords=1000)
records = "\n".join(csw.records.keys())
print("Found {} records.\n".format(len(csw.records.keys())))

In [ ]:
import textwrap

key, value = list(csw.records.items())[-1]
print("\n".join(textwrap.wrap(value.abstract)))

In [ ]:
from geolinks import sniff_link

msg = "geolink: {geolink}\nscheme: {scheme}\nURL: {url}\n".format
for ref in value.references:
    print(msg(geolink=sniff_link(ref["url"]), **ref))

## For more complex examples on how to find data in the catalog please check the IOOS code gallery:
[https://ioos.github.io/notebooks_demos/code_gallery](https://ioos.github.io/notebooks_demos/code_gallery)

## Where to find data?

Curated list of ERDDAP servers: https://github.com/IrishMarineInstitute/awesome-erddap


Environmental Data Service (EDS) model viewer: https://eds.ioos.us


Exploring THREDDS servers: https://unidata.github.io/siphon/latest

## Extras: how does this all work?

## Standards!

![](https://imgs.xkcd.com/comics/standards.png)

## Bad example

In [ ]:
import cftime
import nc_time_axis
from netCDF4 import Dataset

url = "http://goosbrasil.org:8080/pirata/B19s34w.nc"
nc = Dataset(url)

temp = nc["temperature"][:]
times = nc["time"]
temp[temp <= -9999] = np.NaN
t = cftime.num2date(times[:], times.units, calendar=times.calendar)

In [ ]:
fig, ax = plt.subplots()
ax.plot(t, temp[:, 0], ".");

## Good example

In [ ]:
import xarray as xr

ds = xr.open_dataset(url)
temp = ds["temperature"]

In [ ]:
temp.sel(depth_t=1.0, time="2008").plot();